In [118]:
import os
import json
from datetime import datetime, timezone

In [119]:
def year_to_milliseconds(year):
    # Create a datetime object, the format is UTC, is not the same as local.
    start_of_year = datetime(year, 1, 1, 0, 0, 0, tzinfo=timezone.utc) #year, month, day, hour, minute
    
    # Convert the datetime object to milliseconds since Epoch
    milliseconds_since_epoch = int(start_of_year.timestamp() * 1000)
    
    return milliseconds_since_epoch

In [120]:
def get_file_list(root_folder, firstlayer, number_resources):
    """This script will return a list of the files, it searched for the first one and take the next ones depending on the range of the years

    Args:
        root_folder (_type_): _description_
        updates (_type_): _description_

    Returns:
        _type_: _description_
    """
    file_list = []
    append_files = False  # Flag to indicate when to start appending file paths
    folders_processed = 0

    # Iterate over all folders within the main folder
    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name) # build the folder path
        file_path = os.path.join(folder_path, "resource.json")
        # Check if the folder is the "localName" one
        if folder_name == firstlayer:
            file_list.append(file_path)
            append_files = True  # Set the flag to start appending files
            continue  # Move to the next iteration without appending "resource.json" in this folder

        # Append file paths if the flag is set and less than the amount of number_resources
        if append_files and folders_processed < number_resources - 1: # updates["number_resources"]
            file_list.append(file_path)
            folders_processed += 1  # Increment the counter
    
    return file_list

In [121]:
def update_json_files(file_list, year_list, metadata_update):
    """
    Update JSON files in folders named "first_resource_name" within the specified year range.

    Args:
    - file_list (list): list of the files that are input
    - year_list (list): List of the years that are going to be usedame".
    - metadata_update (dict): Dictionary containing the updates to be applied to the JSON files.
    """
    # Store the original title
    original_title = metadata_update["dc:title"]
    
    # they should go in pairs
    for year, file_path in zip(year_list, file_list):
        with open(file_path, 'r') as file:
            try:
                # Load JSON data into a dict
                data = json.load(file)
                
            except json.JSONDecodeError as e:
                # Handle JSON decoding errors
                print(f"Error decoding JSON file {file_path}: {e}")
                continue

        # Build the time string
        tstart = year_to_milliseconds(year)
        tend = year_to_milliseconds(year + 1)
        time_string = "τ1{{tend={},tstart={},ttype=logical}}".format(tend,tstart) # This is the structure: τ1{tend=962409600000,tstart=646790400000,ttype=logical}
        
        """Open the geometry string and update it"""
        geometry_string = data["geometry"]
        # Find the index of the first occurrence of "S2"
        index_of_s2 = geometry_string.find("S2")
        # build the string
        updated_geometry_string = time_string + geometry_string[index_of_s2:]
        # update the field
        data["geometry"] = updated_geometry_string
        
        """Update the rest of the dict with the update values"""
        # Append the year to the title
        metadata_update["dc:title"] = original_title + " " + str(year)
        
        # update the rest
        data["metadata"] = metadata_update
        
        """Little BIG issue, as a dict [] and {} have no space, but the json must. Change the dict to string and replace them"""
        # Transform the dict into a string
        data_string = json.dumps(data, indent=2, separators=(', ', ' : '), ensure_ascii=False)
        # Replace the elements
        data_string = data_string.replace("[]","[ ]").replace("{}","{ }")

        """Uncoment this part once you are sure what you are doing"""
        # Open the JSON file for writing
        with open(file_path, 'w') as file:
            file.write(data_string)
            
            
    return data, print("Finish")


In [122]:
root_folder = r"C:\Users\ruben.crespo\.klab\workspace\esa.people.sandbox\resources"
firstlayer = "esa_people.copernicus_combined_drought_indicator_avg_of_eu_v3_2012" #name of the folder

"""These two have to match"""
number_resources = 11
year_list = range(2012, 2022 + 1, 1) # or give a set of unique years [2001, 2002]


"""Define the content""" # - \r\n = Enter
Title = "Combined Drought Indicator (CDI) v3"
Description = "The CDI is an indicator for drought early warning, specifically designed to monitor agricultural\n drought. Through the combination of spatial patterns of precipitation, soil moisture and greenness\n vegetation anomalies, the CDI identifies areas at risk of agricultural drought, areas where the\n vegetation has already been affected by drought and areas in the process of recovery to normal\n conditions. Accordingly, the CDI classification scheme defines three primary drought classes\n (“Watch”, “Warning” and “Alert”) and three recovery classes (“Temporary Soil Moisture recovery”,\n “Temporary vegetation recovery” and “Recovery”). The CDI version 2.0.0 (hereafter, v2) has been\n developed by Cammalleri et al. (2021), following the work of Sepulcre-Canto et al. (2012). The\n European Drought Observatory released the CDI v3 update in March 2023. This revision mainly\n benefits from the incorporation of time-varying crop masks, which prevent the use of the ALERT\n impact class (stress for vegetation) in crop areas out of the growing season.\n\n The resource has been adapted transforming the classes from 0 to 3, 0 meaning no drought, 1 watch,\n 2 warning and 3 Alert, into percentages:\n 0 = 1.0; 1 = 0.66; 2 = 0.33 and 3 = 0\n later, the average of all the 36 of all the timestamps for a pixel over the year."
Originating_institution = "Copernicus European Drought Observatory (EDO)"
Url_doi = "https://joint-research-centre.ec.europa.eu/system/files/2023-09/Copernicus_EDO_Factsheet_CDI_v3_20230515.pdf"
Autors = "Rubén Crespo Ceballos"
Theme = "Soil"
Geographical_region = "Global"
Keywords = "Drought severity"
References = """Cammalleri, C. and Arias-Muñoz, C. and Barbosa, P. and de Jager, A. and Magni, D. and Masante, 
D. and Mazzeschi, M. and McCormick, N. and Naumann, G. and Spinoni, J. and and Vogt, J. 2021. 
A revision of the Combined Drought Indicator (CDI) used in the European Drought Observatory
(EDO), Nat. Hazards Earth Syst. Sci., 21, 481–495. 

Sepulcre-Canto, G. and Horion, S. and Singleton, A. and Carrão, H. and Vogt, J. 2012. 
Development of a Combined Drought Indicator to detect agricultural drought in Europe. Natural 
Hazards and Earth System Sciences, 12, 3519-3531."""
Notes = ""

metadata_update = {
    "im:keywords" : Keywords,
    "dc:comment" : Description,
    "im:notes" : Notes,
    "dc:title" : Title,
    "dc:url" : Url_doi,
    "dc:creator" : Autors,
    "im:thematic-area" : Theme,
    "dc:originator" : Originating_institution,
    "im:geographic-area" : Geographical_region,
    "dc:source" : References
    }
file_list = get_file_list(root_folder, firstlayer, number_resources)
# Make sure you are changing what you want
print(file_list[:])



['C:\\Users\\ruben.crespo\\.klab\\workspace\\esa.people.sandbox\\resources\\esa_people.copernicus_combined_drought_indicator_avg_of_eu_v3_2012\\resource.json', 'C:\\Users\\ruben.crespo\\.klab\\workspace\\esa.people.sandbox\\resources\\esa_people.copernicus_combined_drought_indicator_avg_of_eu_v3_2013\\resource.json', 'C:\\Users\\ruben.crespo\\.klab\\workspace\\esa.people.sandbox\\resources\\esa_people.copernicus_combined_drought_indicator_avg_of_eu_v3_2014\\resource.json', 'C:\\Users\\ruben.crespo\\.klab\\workspace\\esa.people.sandbox\\resources\\esa_people.copernicus_combined_drought_indicator_avg_of_eu_v3_2015\\resource.json', 'C:\\Users\\ruben.crespo\\.klab\\workspace\\esa.people.sandbox\\resources\\esa_people.copernicus_combined_drought_indicator_avg_of_eu_v3_2016\\resource.json', 'C:\\Users\\ruben.crespo\\.klab\\workspace\\esa.people.sandbox\\resources\\esa_people.copernicus_combined_drought_indicator_avg_of_eu_v3_2017\\resource.json', 'C:\\Users\\ruben.crespo\\.klab\\workspace\\e

In [123]:
for file in file_list:
    print(file)

C:\Users\ruben.crespo\.klab\workspace\esa.people.sandbox\resources\esa_people.copernicus_combined_drought_indicator_avg_of_eu_v3_2012\resource.json
C:\Users\ruben.crespo\.klab\workspace\esa.people.sandbox\resources\esa_people.copernicus_combined_drought_indicator_avg_of_eu_v3_2013\resource.json
C:\Users\ruben.crespo\.klab\workspace\esa.people.sandbox\resources\esa_people.copernicus_combined_drought_indicator_avg_of_eu_v3_2014\resource.json
C:\Users\ruben.crespo\.klab\workspace\esa.people.sandbox\resources\esa_people.copernicus_combined_drought_indicator_avg_of_eu_v3_2015\resource.json
C:\Users\ruben.crespo\.klab\workspace\esa.people.sandbox\resources\esa_people.copernicus_combined_drought_indicator_avg_of_eu_v3_2016\resource.json
C:\Users\ruben.crespo\.klab\workspace\esa.people.sandbox\resources\esa_people.copernicus_combined_drought_indicator_avg_of_eu_v3_2017\resource.json
C:\Users\ruben.crespo\.klab\workspace\esa.people.sandbox\resources\esa_people.copernicus_combined_drought_indica

In [124]:
data_1 = update_json_files(file_list[:], year_list[:], metadata_update) # for testing

Finish


In [59]:
print(data_1)

({'urn': 'local:belen.inguanzo:esa.people.sandbox:esa_people.copernicus_combined_drought_indicator_avg_of_eu_v3_2022', 'version': '0.0.1', 'adapterType': 'wcs', 'localPath': 'esa.aries.peopleea/resources/esa_people.copernicus_combined_drought_indicator_avg_of_eu_v3_2022', 'geometry': 'τ1{tend=1672531200000,tstart=1640995200000,ttype=logical}S2(999,949){bbox=[-35.116786562537584 73.99983360423174 23.912106785228683 72.61731301863757],proj=EPSG:4326}', 'projectName': 'esa.aries.peopleea', 'localName': 'esa_people.copernicus_combined_drought_indicator_avg_of_eu_v3_2022', 'type': 'NUMBER', 'resourceTimestamp': 1702887434713, 'metadata': {'im:keywords': 'Drought severity', 'dc:comment': 'The CDI is an indicator for drought early warning, specifically designed to monitor agricultural\n drought. Through the combination of spatial patterns of precipitation, soil moisture and greenness\n vegetation anomalies, the CDI identifies areas at risk of agricultural drought, areas where the\n vegetation